# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
va_city_data_df = pd.read_csv("../Output_data/cities.csv")
va_city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Check datatypes for lat and lng
locations = va_city_data_df[["Lat", "Lng"]].astype(float)
locations.dtypes

Lat    float64
Lng    float64
dtype: object

In [5]:
# Check datatypes for humidity
humidity = va_city_data_df["Humidity"]
humidity.dtypes

dtype('int64')

In [6]:
# Create heatmap layer for locations and humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Create dataframe with only the columns we need (drop date column)
va_data_df = va_city_data_df[["City","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country"]]
va_data_df.head()

#narrow down criteria 
narrow_va_data_df = va_data_df[(va_data_df["Max Temp"]>72) & (va_data_df["Max Temp"]<78) &
                                   (va_data_df["Humidity"]<80) & (va_data_df["Cloudiness"]<25) &
                                   (va_data_df["Wind Speed"]<5)]

narrow_va_data_df                                    

#drop any rows with null values 
cleaned_va_data = narrow_va_data_df.dropna(how="any")
cleaned_va_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
153,porto novo,-23.68,-45.44,77.00,53,20,4.70,BR
297,mogok,22.92,96.51,72.24,51,0,2.77,MM
494,baylo,7.85,126.06,77.00,69,1,1.07,PH


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Drop any rows with null values 
cleaned_va_data = narrow_va_data_df.dropna(how="any")
cleaned_va_data

# Create a new dataframe for hotel info
hotel_df = cleaned_va_data

# Add hotel name column 
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
153,porto novo,-23.68,-45.44,77.00,53,20,4.70,BR,
297,mogok,22.92,96.51,72.24,51,0,2.77,MM,
494,baylo,7.85,126.06,77.00,69,1,1.07,PH,


In [9]:
# Create a new dataframe for hotel info
hotel_df = cleaned_va_data

# Add hotel name column 
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
153,porto novo,-23.68,-45.44,77.00,53,20,4.70,BR,
297,mogok,22.92,96.51,72.24,51,0,2.77,MM,
494,baylo,7.85,126.06,77.00,69,1,1.07,PH,


In [10]:
# Define radius and type 
hotels = "lodging"
radius = 5000

# Set parameters
params = {
    "types": hotels,
    "radius": radius,
    "key": g_key
}

# Iterate through hotel_df: 
for index, row in hotel_df.iterrows():
    
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #run request
    data = requests.get(base_url, params=params).json()

    results = data["results"]
    
    try: 
        hotel_name = data["results"][0]["name"]
        print(f"Retrieving Results for Index {index}: {hotel_name}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotel_name

    except (KeyError, IndexError): 
        print("Cannot find hotel for", row["City Name"])
        
    print("------------------")

Retrieving Results for Index 153: Harmonia.
------------------
Retrieving Results for Index 297: MOGOK HILL HOTEL.
------------------
Retrieving Results for Index 494: PJRB Inland resort.
------------------


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer 
marker_layer = gmaps.symbol_layer(locations, fill_color ='green',
                                  stroke_color='navy', info_box_content=hotel_info)
# fig = gmaps.figure()
# fig.add_layer(marker_layer)
# fig

In [18]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))